# Use BoTorch for feasibility weighted acquisition function

import libearies

In [1]:
import pandas as pd
import numpy as np
import torch
import botorch
import gpytorch
import matplotlib.pyplot as plt

#modules for regression
from botorch.models.gp_regression import SingleTaskGP
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from gpytorch.mlls.sum_marginal_log_likelihood import ExactMarginalLogLikelihood
from botorch.utils.transforms import unnormalize, normalize

#modules for BO
from botorch.optim.optimize import optimize_acqf, optimize_acqf_list
from botorch.acquisition.objective import GenericMCObjective
from botorch.utils.multi_objective.scalarization import get_chebyshev_scalarization
from botorch.utils.multi_objective.box_decompositions.non_dominated import (
    FastNondominatedPartitioning,
)
from botorch.acquisition.multi_objective.monte_carlo import (
    qExpectedHypervolumeImprovement,
    qNoisyExpectedHypervolumeImprovement,
)
from botorch.utils.sampling import sample_simplex
from botorch import fit_gpytorch_mll
from botorch.exceptions import BadInitialCandidatesWarning
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.utils.multi_objective.box_decompositions.dominated import (
    DominatedPartitioning,
)
from botorch.utils.multi_objective.pareto import is_non_dominated

from gpytorch.likelihoods import DirichletClassificationLikelihood
from botorch.acquisition.multi_objective.objective import IdentityMCMultiOutputObjective


c:\Users\goura\anaconda3\envs\botorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Read and normalize data

In [2]:
filename = r'../data/olhs_combine.xlsx'
x_pd = pd.read_excel(filename, sheet_name='Design', header=[0,1], index_col=[0])
y_pd = pd.read_excel(filename, sheet_name='bo_data', header=[0,1], index_col=[0])

dtype=torch.double

objective_properties = ['Polymer Solubility', 'Gelation Enthalpy', 'Shear Modulus']

x = torch.tensor(x_pd.values, dtype=dtype)
y = torch.tensor(y_pd[objective_properties].values, dtype=dtype)
mfg = torch.tensor(y_pd['Manufacturability'].values, dtype=torch.long)

x_bounds = np.array([[2000, 10000], [0, 100], [0, 40], [5000, 15000], [80, 100], [0,100], [60, 100], [70, 100]])
x_bounds = torch.tensor(x_bounds.T, dtype=dtype)

x = normalize(x, bounds=x_bounds)

In [3]:
# identify non-feasible designs
feas_bool = mfg==1
x_reg = x[feas_bool.flatten(), :] 
y_reg = y[feas_bool.flatten(), :]

Helper functions

Define and initialize regression and classification model

In [4]:
# Regression model
models = []
for i in range(len(objective_properties)):
    models.append(SingleTaskGP(x_reg, y_reg[:, i].unsqueeze(-1), 
                               outcome_transform=Standardize(m=1)))

# transformation of mfg labels
tmp_likl = DirichletClassificationLikelihood(targets=mfg.squeeze(), alpha_epsilon=1e-4, 
                                             learn_additional_noise=False)
cls_model = SingleTaskGP(train_X=x, train_Y=-tmp_likl.transformed_targets.T.double(), 
                         outcome_transform=Standardize(m=2))

models.append(cls_model)

model = ModelListGP(*models)
mll = SumMarginalLogLikelihood(model.likelihood, model)
mll = mll.to(x)

Fit the models to data

In [5]:
# regression model
fit_gpytorch_mll(mll)

SumMarginalLogLikelihood(
  (likelihood): LikelihoodList(
    (likelihoods): ModuleList(
      (0-3): 4 x GaussianLikelihood(
        (noise_covar): HomoskedasticNoise(
          (noise_prior): GammaPrior()
          (raw_noise_constraint): GreaterThan(1.000E-04)
        )
      )
    )
  )
  (model): ModelListGP(
    (models): ModuleList(
      (0-3): 4 x SingleTaskGP(
        (likelihood): GaussianLikelihood(
          (noise_covar): HomoskedasticNoise(
            (noise_prior): GammaPrior()
            (raw_noise_constraint): GreaterThan(1.000E-04)
          )
        )
        (mean_module): ConstantMean()
        (covar_module): ScaleKernel(
          (base_kernel): MaternKernel(
            (lengthscale_prior): GammaPrior()
            (raw_lengthscale_constraint): Positive()
          )
          (outputscale_prior): GammaPrior()
          (raw_outputscale_constraint): Positive()
        )
        (outcome_transform): Standardize()
      )
    )
    (likelihood): LikelihoodList

Prediction for sanity check

In [6]:
model.eval()

with torch.no_grad():
    posterior = model.posterior(x)
    pred_mean = posterior.mean
pred_mean

tensor([[5.6105e+01, 3.2461e+00, 5.5199e+00, 1.3639e+01, 5.2274e-01],
        [2.2391e+02, 2.1208e+01, 1.6374e+01, 1.1603e+00, 1.3002e+01],
        [1.6535e+02, 2.1728e+01, 1.6213e+01, 1.2017e+00, 1.2960e+01],
        [1.8709e+02, 2.4024e+01, 1.3192e+01, 1.0066e+00, 1.3155e+01],
        [1.2611e+02, 2.0011e+00, 3.0697e+00, 1.3694e+01, 4.6791e-01],
        [8.9072e+01, 8.2121e+01, 2.5984e+00, 1.3672e+01, 4.9007e-01],
        [9.0486e+01, 2.4459e+00, 3.4244e+00, 1.3590e+01, 5.7192e-01],
        [7.0479e+01, 2.7199e+01, 4.3013e+00, 1.3638e+01, 5.2377e-01],
        [5.4015e+01, 1.0730e+02, 5.7114e+00, 1.3650e+01, 5.1176e-01],
        [1.7547e+02, 3.7080e+00, 1.0451e+01, 1.3692e+01, 4.6988e-01],
        [5.4411e+01, 1.1299e+02, 1.4331e+00, 1.3527e+01, 6.3491e-01],
        [1.0590e+02, 1.7457e+01, 3.1507e+00, 1.3655e+01, 5.0666e-01],
        [8.4093e+01, 9.1354e+00, 1.0484e+01, 1.3645e+01, 5.1683e-01],
        [7.0150e+01, 1.0539e+02, 3.9058e+01, 1.3589e+01, 5.7307e-01],
        [3.3197e+02,

Helper functions for BO

In [7]:
bounds = torch.zeros(2, 8)
bounds[1] = 1

BATCH_SIZE = 4      # Number of candidates selected in each BO run/iteration
NUM_RESTARTS = 10   # Restarts during BO run
RAW_SAMPLES = 512   

ref_point = ref_point = torch.tensor([18, 0.1, 0.01], dtype=dtype)

In [8]:
from typing import Optional

def obj_callable(Z: torch.Tensor, X: Optional[torch.Tensor] = None):
    return IdentityMCMultiOutputObjective([0,1,2])

def constraint_callable(Z):
    class0 = Z[..., -2]
    class1 = Z[..., -1]
    return class1 - class0 #neg for feasiblle values

In [9]:
def optimize_qnehvi_and_get_observation(model, train_x, train_obj, sampler):
    """Optimizes the qEHVI acquisition function, and returns a new candidate and observation."""
    with torch.no_grad():
        pred = model.posterior(normalize(train_x, x_bounds)).mean

    acq_func = qNoisyExpectedHypervolumeImprovement(
        model=model,
        ref_point=ref_point,
        X_baseline=train_x,
        sampler=sampler,
        prune_baseline=True,
        objective=IdentityMCMultiOutputObjective(outcomes=[0, 1, 2]),
        constraints=[constraint_callable],
    )

    #optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,
        options={"batch_limit":5, "maxiter": 200},
        sequential=True,
    )

    new_x = unnormalize(candidates.detach(), bounds=x_bounds)
    return new_x
    

In [10]:
sampler = SobolQMCNormalSampler(sample_shape=torch.Size([128]))

new_x_qnehvi = optimize_qnehvi_and_get_observation(model, x, y, sampler)

In [11]:
new_x_qnehvi

tensor([[4.5400e+03, 6.6770e+01, 2.9999e+01, 1.4463e+04, 9.8290e+01, 8.4361e+00,
         8.7870e+01, 8.7271e+01],
        [4.6462e+03, 5.1599e+01, 2.2852e+01, 1.4436e+04, 9.2162e+01, 6.4709e+01,
         9.3740e+01, 8.5816e+01],
        [4.5897e+03, 6.1354e+01, 2.6945e+01, 1.4385e+04, 9.6520e+01, 3.1376e+01,
         9.0703e+01, 8.6327e+01],
        [5.0443e+03, 0.0000e+00, 2.4051e+01, 1.4235e+04, 8.7934e+01, 7.5644e+01,
         9.2554e+01, 9.2086e+01]], dtype=torch.float64)

In [12]:
model.posterior(new_x_qnehvi).mean

tensor([[170.7780,  24.1824,  13.7389,  11.1271,   3.0349],
        [170.7780,  24.1824,  13.7389,  11.1271,   3.0349],
        [170.7780,  24.1824,  13.7389,  11.1271,   3.0349],
        [170.7780,  24.1824,  13.7389,  11.1271,   3.0349]],
       dtype=torch.float64, grad_fn=<CloneBackward0>)

In [13]:
cls_likl = DirichletClassificationLikelihood(targets=mfg.squeeze(), alpha_epsilon=1e-4, learn_additional_noise=False)
cls_model = SingleTaskGP(train_X=x, train_Y=cls_likl.transformed_targets.T.double(), outcome_transform=Standardize(m=2))
mll_cls = ExactMarginalLogLikelihood(cls_model.likelihood, cls_model)
fit_gpytorch_mll(mll_cls)

cls_model.eval()
with torch.no_grad():
    post_cls = cls_model.posterior(new_x_qnehvi)
samples_cls = post_cls.sample(torch.Size((256,))).exp()
probabilities = (samples_cls / samples_cls.sum(-1, keepdim=True)).mean(0)
probabilities

tensor([[0.1588, 0.8412],
        [0.1546, 0.8454],
        [0.1302, 0.8698],
        [0.1497, 0.8503]], dtype=torch.float64)